## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling.

Для начала, установим Yandex Cloud ML SDK. В идеальном мире, вы сделаете вот так:

In [9]:
%pip install --upgrade yandex-cloud-ml-sdk

I0000 00:00:1742470000.768423    3331 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: yandex-cloud-ml-sdk
    Found existing installation: yandex-cloud-ml-sdk 0.4.1
    Uninstalling yandex-cloud-ml-sdk-0.4.1:
      Successfully uninstalled yandex-cloud-ml-sdk-0.4.1

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


В нашем случае мы установим самую последнюю версию SDK:

In [6]:
%pip install flit
%pip install git+https://github.com/yandex-cloud/yandex-cloud-ml-sdk@5d77a3618838cc4ca0553a777a39a542f900e7c8

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/yandex-cloud/yandex-cloud-ml-sdk (to revision 5d77a3618838cc4ca0553a777a39a542f900e7c8) to /tmp/pip-req-build-fuvbe4ro
  Running command git clone --filter=blob:none --quiet https://github.com/yandex-cloud/yandex-cloud-ml-sdk /tmp/pip-req-build-fuvbe4ro
  Running command git rev-parse -q --verify 'sha^5d77a3618838cc4ca0553a777a39a542f900e7c8'
  Running command git fetch -q https://github.com/yandex-cloud/yandex-cloud-ml-sdk 5d77a3618838cc4ca0553a777a39a542f900e7c8
  Running command git checkout -q 5d77a3618838cc4ca0553a777a39a542f900e7c8
  Resolved https://github.com/yandex-cloud/yandex-cloud-ml-sdk to commit 5d77a3618838cc4ca0553a777a39a542f900e7c8
  Preparing metadata (pyproject.t

Также необходимо обновить некоторые библиотеки:

In [53]:
%pip install --upgrade pydantic

I0000 00:00:1743425416.188163    3524 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.1.0 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.
inflect 6.0.5 requires pydantic<2,>=1.9.1, but you have pydantic 2.11.1 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
thinc 8.1.10 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] T

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere.

Создадим модель последней версии YandexGPT 5 и убедимся, что она кое-что знает про вина:

In [1]:
import os 
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ['folder_id']
api_key = os.environ['api_key']

sdk = YCloudML(folder_id=folder_id,auth=api_key)
model = sdk.models.completions("yandexgpt",model_version='rc')

In [8]:
model.run("Какое вино можно пить с устрицами?").text

'Выбор вина к устрицам зависит от ваших личных предпочтений и вида устриц. Вот несколько популярных вариантов:\n\n1. **Шардоне** — особенно из прохладных климатических зон, таких как Шабли во Франции или некоторые регионы Новой Зеландии. Эти вина обычно имеют яркую кислотность и ноты цитрусовых фруктов, которые хорошо сочетаются с солоноватостью и текстурой устриц.\n\n2. **Совиньон Блан** — особенно из регионов, таких как Марльборо в Новой Зеландии или Пойяк во Франции. Совиньон Блан обладает яркой кислотностью и свежими фруктовыми нотами, которые могут подчеркнуть вкус устриц.\n\n3. **Мюскаде** — белое вино из региона Луары во Франции, которое часто рекомендуют к устрицам. Оно обладает лёгким, свежим вкусом с нотами зелёного яблока и цитрусовых.\n\n4. **Альбариньо** — белое вино из региона Риас Байшас в Испании. Оно известно своей свежестью, высокой кислотностью и нотами экзотических фруктов, которые могут хорошо сочетаться с устрицами.\n\n5. **Шампанское и другие игристые вина** — их

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [35]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy="static")

def create_assistant(model,tools=None):
    if tools and len(tools)>0:
        return sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active",tools=tools)
    else:
        return sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active")

In [37]:
thread = create_thread()
assistant =  create_assistant(model)

assistant.update(instruction="Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.")

thread.write("Привет! Какое вино посоветуете?")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Здравствуйте! Чтобы подобрать вино, которое максимально соответствует вашим предпочтениям, мне нужно узнать немного больше о ваших вкусах. Вы предпочитаете белые, розовые или красные вина? Есть ли у вас любимые сорта винограда или страны-производители? Также важно знать, планируете ли вы подавать вино к определённому блюду или событию.


In [38]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Для стейка я бы порекомендовал красное вино с насыщенным вкусом и хорошей структурой. Отлично подойдут вина из сортов каберне совиньон, мерло или шираз. Эти сорта обладают достаточной танинностью и фруктовостью, чтобы гармонично сочетаться с мясом.

Если вы предпочитаете более лёгкие и фруктовые вина, то обратите внимание на пино нуар — он также хорошо подойдёт к стейку.

Рекомендую обратить внимание на вина из следующих регионов:
* Бордо (Франция) — каберне совиньон;
* Тоскана (Италия) — мерло;
* Австралия — шираз;
* Бургундия (Франция) — пино нуар.

Надеюсь, это поможет вам сделать выбор! Если у вас есть ещё вопросы или нужна дополнительная информация, пожалуйста, сообщите мне.


In [39]:
thread.delete()
assistant.delete()

## Добавляем RAG

Для RAG будем использовать текстовую базу знаний по винам и винным регионам, которая хранится в виде множества файлов в директориях `source/wines` и `source/regions`. Пройдёмся по этим файлам и посмотрим на их длину в токенах.

In [3]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
    with open(filename,'r',encoding='utf8') as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename,encoding='utf-8') as f:
        l = len(f.read())
    return l
    
d = [
        {
            "File" : fn,
            "Tokens" : get_token_count(fn),
            "Chars" : get_file_len(fn),
            "Category" : fn.split('/')[1]
        } 
    for fn in glob('source/*/*.md')
    if fn.count('/') == 2
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars,Category
0,source/regions/Абруццо.md,499,2022,regions
1,source/regions/Азорские острова.md,409,1809,regions
2,source/regions/Аконкагуа.md,278,1182,regions
3,source/regions/Алентежу.md,320,1216,regions
4,source/regions/Апулия.md,489,1918,regions
...,...,...,...,...
125,source/wines/Совиньон блан.md,634,2578,wines
126,source/wines/Темпранильо.md,637,2323,wines
127,source/wines/Цвайгельт.md,663,2620,wines
128,source/wines/Шардоне.md,567,2407,wines


Посмотрим на среднюю, мин и макс длину фрагментов:

In [4]:
df.groupby('Category').agg({ 'Tokens' : ('min','mean','max') })

Tokens                 
            min        mean  max
Category                        
regions     153  423.340000  683
wines       264  551.366667  664

Мы видим, что фрагменты не превышают 700 токенов, и это значит, что нам не придётся прибегать к какой-либо стратегии чанкования. Это идеальная ситуация, когда текстовая база знаний разбита вручную на небольшие фрагменты текста.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [5]:
def upload_file(filename):
    return sdk.files.upload(filename,ttl_days=1,expiration_policy="static")

df["Uploaded"] = df['File'].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и вектороного поиска.

Также в процессе создания индекса мы указываем **стратегию чанкования** текстов, но в нашем случае она не будет применяться.

> Поскольку есть ограничение на 100 добавляемых в индекс файлов, то будем добавлять фрагменты по винам и по регионам по-очереди.

In [6]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy)

op = sdk.search_indexes.create_deferred(
    df[df["Category"]=="wines"]["Uploaded"],
    index_type = HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens = 1000,
            chunk_overlap_tokens = 100),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy()))
index = op.wait()

In [7]:
op = index.add_files_deferred(df[df["Category"]=="regions"]["Uploaded"])
xfiles = op.wait()

## Собираем RAG-ассистента

Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [8]:
search_tool = sdk.tools.search_index(index)

assistant = create_assistant(model, tools=[search_tool])
thread = create_thread()

instruction="""
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

_ = assistant.update(instruction=instruction)

In [9]:
thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)

К стейку подойдут следующие вина:

1. **Из Сицилии:** черасуоло ди виттория (Cerasuolo di Vittoria) — гармонирует с мясом на вертеле и тушёной говядиной.
2. **Из Франшхука:** красные вина из сортов каберне совиньон, шираз и мерло, а также пинотаж — универсальное вино, которое хорошо сочетается с перчёными стейками из говядины и баранины.
3. **Из Орегона:** пино нуар — хорошо подойдёт к стейку из лосося и красному мясу, например, говядине по-бургундски.
4. **Из долины Маллеко (Чили):** красное сухое вино — сочетается со стейком на гриле.
5. **Сира или шираз:** особенно хорошо дополняют блюда из красного мяса, например, стейки из баранины или говядины.


Посмотрим, из каких источников был получен этот ответ:

In [10]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != 'filechunk':
                continue
            print('------------------------')
            print(source.parts[0])
            
print_citations(result)

------------------------
## Сицилия 

Сицилия
Сицилия — регион на юге Италии. В его состав входит одноименный и самый большой в Средиземном море остров и несколько более мелких островов рядом с ним. На восточном побережье Сицилии возвышается Этна — самый высокий действующий вулкан в Европе. Помимо итальянского в регионе говорят на сицилийском языке. Административный центр Сицилии — Палермо. Это самый южный из винодельческих регионов Италии. Здесь около 120 тысяч га виноградников. С начала нулевых на Сицилии бурно развивается производство вулканических вин. Их производят из винограда, выросшего у подножия Этны. Пепел самого высокого действующего вулкана Европы богат микроэлементами, поэтому местные почвы очень плодородны. И, главное, в таких условиях нет шанса у филлоксеры. К наиболее значимым местным сортам относят красный неро д'авола. Также на Сицилии популярны красные гренаш, перриконе и ночера. Из белых стоит отметить катаратто — его смешивают с сортами грилло и инзолия и производя

In [11]:
thread.delete()

Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `source/food_wine_table.md` в формате markdown.

In [12]:
with open('source/food_wine_table.md',encoding='utf-8') as f:
    food_wine = f.readlines()
fw = ''.join(food_wine)

tokens = len(model.tokenize(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 12630, 3.3103721298495645 chars/token


Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [13]:
header=food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [14]:
chunk_size = 600 * 3 # approx 600 tokens * 2 char/token

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
    s.append(x)
    if len(''.join(s))>chunk_size:
        id = sdk.files.upload_bytes(''.join(s).encode(),ttl_days=5,expiration_policy="static")
        uploaded_foodwine.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_foodwine)} table chunks")

Uploaded 22 table chunks


Теперь добавим эти фрагменты в индекс:

In [15]:
op = index.add_files_deferred(uploaded_foodwine)
xfiles = op.wait()

Посмотрим, стал ли ответ системы лучше:

In [16]:
thread = create_thread()

thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)
print_citations(result)

Выбор вина к стейку зависит от его вида и степени прожарки.

1. Для нежного мраморного стейка (например, филе-миньон) подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

2. Для жирноватого стейка (например, рибай) с прожаркой Rare рекомендуются выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки.

3. Для прожарки Medium или WellDone к жирноватому стейку подойдут сухие и полусухие вина из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и в

In [17]:
thread.delete()
assistant.delete()

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [18]:
import pandas as pd

pl = pd.read_excel('source/wine-price.xlsx')
pl

,Номер артикула,название,CT,цена от 1 бутылки,от 3-х и более,регулярная цена/промо
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены
...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN


Для удобства переименуем колонки:

In [19]:
pl.columns=['Id','Name','Country','Price','WHPrice','etc']

Из имени извлечём кислотность вина:

In [20]:
acid_map = { 'СХ' : 'Сухое', 'СЛ' : 'Сладкое', 'ПСХ' : 'Полусухое', 'ПСЛ' : 'Полусладкое' }
pl['Acidity'] = pl['Name'].apply(lambda x: acid_map.get(x.split()[-1].replace('КР',''),''))
pl['Acidity'].value_counts()

Сухое          614
Полусухое       68
Полусладкое     28
Сладкое         23
                19
Name: Acidity, dtype: int64

Тоже самое сделаем с цветом:

In [21]:
pl['Color'] = pl['Name'].apply(lambda x: 'Красное' if (x.split()[-1].startswith("КР") or x.split()[-2]=="КР") else '')
pl['Color'].value_counts()

Красное    739
            13
Name: Color, dtype: int64

В итоге получилась такая таблица вин:

In [22]:
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN,Сухое,Красное
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN,Сухое,Красное
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN,Сухое,Красное
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN,Сухое,Красное
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены,Сухое,Красное
...,...,...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены,Сладкое,Красное
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN,Сухое,Красное


Чтобы научиться отвечать на вопросы по прайс-листу, у нас есть несколько возможных решений:

* Попытаться закинуть прайс-лист в контекст модели с помощью RAG, как мы это ранее делали с таблицей блюд. Не очень хороший вариант, потому что модель не будет видеть всю таблицу, и не сможет отвечать на вопросы "максимальная/минимальная цена".
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

ML SDK позволяет нам упростить function calling, и вместо JSON-схемы использовать типизированные объекты Pyton. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [23]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам.""" 
    name : str = Field(description="Название вина",default=None)
    country : str = Field(description="Страна",default=None)
    acidity : str = Field(description="Кислотность (сухое, полусухое, сладкое, полусладкое)",default=None)
    color : str = Field(description="Цвет вина (красное, белое, розовое)",default=None)
    sort_order : str = Field(description="Порядок выдачи (most expensive, cheapest, random, average)",default=None)
    what_to_return : str = Field(description="Что вернуть (wine info или price)",default=None)

Теперь создадим инструмент (tool) и нового ассистента, у которого в списке инструментов будет одновременно и RAG-поиск, и function calling. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [24]:
price_list_search_tool = sdk.tools.function(SearchWinePriceList)

assistant = create_assistant(model,tools=[price_list_search_tool,search_tool])
thread = create_thread()

instruction="""
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин
или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

_ = assistant.update(instruction=instruction)

Попробуем узнать самое дешевое вино из Австралии:

In [28]:
thread.write("Привет! Какое самое дешевое красное сухое вино из Австралии?")
run = assistant.run(thread)
res = run.wait()
res

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='SearchWinePriceList', arguments={'what_to_return': 'price', 'acidity': 'сухое', 'color': 'красное', 'sort_order': 'cheapest', 'country': 'Австралия'})),), _message=None, usage=Usage(input_text_tokens=2695, completion_tokens=39, total_tokens=2734))

Видим, что в итоге вернулся ответ, указывающий, что надо сделать Funtion Call для получения данных из таблицы. При это LLM извлекла из текста все параметры запроса.

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта `SearchWinePriceList`:

In [90]:
country_map = {
    'IT' : 'Италия', 'FR' : 'Франция', 'ES' : 'Испания',
    'RU' : 'Россия', 'PT' : 'Португалия', 'AR' : 'Армения',
    'CL' : 'Чили', 'AU' : 'Австрия', 'GE' : 'Грузия',
    'ZA' : 'ЮАР', 'US' : 'США', 'NZ' : 'Новая Зеландия',
    'DE' : 'Германия', 'AT' : 'Австрия', 'IL' : 'Израиль',
    'BG' : 'Болгария', 'GR' : 'Греция', 'AU' : 'Австралия' }

revmap = { v.lower() : k for k,v in country_map.items() }

def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x['Country']==revmap[req.country.lower()]]
    if req.acidity:
        x = x[x['Acidity']==req.acidity.capitalize()]
    if req.color:
        x = x[x['Color']==req.color.capitalize()]
    if req.name:
        x = x[x['Name'].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order:
        if req.sort_order=='cheapest':
            x = x.sort_values(by='Price')
        elif req.sort_order=='most expensive':
            x = x.sort_values(by='Price',ascending=False)
        else:
            pass
    if x is None or len(x)==0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + '\n'.join([
        f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}" for _,z in x.head(10).iterrows()
    ])

print(find_wines(SearchWinePriceList(country='Австралия',sort_order='cheapest')))

Вот какие вина были найдены:
0,75ВИНО ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
0,75ВИНО ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
0,75ВИНО ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
0,75ВИНО ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
0,75ВИНО ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
0,75ВИНО СТАМП ДЖАМП КР СХ (Австралия) - 789.0
0,75ВИНО ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
0,75ВИНО ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
0,75ВИНО СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
0,75ВИНО ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


В ответ на Function Call нам нужно сформировать ответ от функции, и передать её обратно для обработки языковой моделью. В нашем случае функция может быть только одна, поэтому мы не проверяем название функции, а всегда запрашиваем поиск в таблице:

In [29]:
import time

if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Processing function call fn={f.function.name}")
        x = SearchWinePriceList.model_validate(f.function.arguments)
        x = find_wines(x)
        result.append({ "name" : f.function.name, "content"  : x })
    run.submit_tool_results(result)
    time.sleep(3)
    res = run.wait()
res

 + Processing function call fn=SearchWinePriceList


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvt5281i43i9ih2044th', parts=('Самое дешёвое красное сухое вино из Австралии в нашем магазине — это «ЧОЛК ХИЛЛ ШИРАЗ КР СХ» по цене 509.0 рублей за бутылку объёмом 0,75 литра.',), thread_id='fvth4ofmn5bnhjq0cl74', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 4, 1, 11, 19, 56, 708394), labels=None, author=Author(id='fvto8lhjhbjfni037od0', role='ASSISTANT'), citations=()), usage=Usage(input_text_tokens=3386, completion_tokens=83, total_tokens=3469))

In [30]:
thread.delete()
assistant.delete()

## Релизуем агента с Function Calling

Для упрощения реализации Function Calling напишем небольшую обвязку, реализующую агента, способного искать в текствой базе и делать Function Calling. Для описания функциональности агента нам нужно будет передать ему таблицу из функций.

In [116]:
class Agent:
    def __init__(self,assistant=None,instruction=None, search_index=None, tools=None):
        
        self.tools = tools
        self.thread = None
        
        if assistant:
            self.assistant = assistant
        else:
            if tools:
                tools = [ sdk.tools.function(k) for k,v in tools.items() ]
            else:
                tools = []
            if search_index:
                tools.append(sdk.tools.search_index(search_index))
            self.assistant = create_assistant(model,tools)
        
        if instruction:
            self.assistant.update(instruction=instruction)
                    
    def get_thread(self,thread=None):
        if thread is not None:
            return thread
        if self.thread == None:
            self.thread = create_thread()
        return self.thread
                
    def __call__(self, message, thread=None):
        thread = self.get_thread(thread)
        thread.write(message)
        run = self.assistant.run(thread)
        res = run.wait()
        if res.tool_calls:
            result = []
            for f in res.tool_calls:
                fn = eval(f.function.name)
                args = fn.model_validate(f.function.arguments)
                print(f"Вызываем функцию {f.function.name}, args={args}")
                x = self.tools[fn](args,thread)
                result.append({ "name" : f.function.name, "content"  : x })
            run.submit_tool_results(result)
            time.sleep(3)
            res = run.wait()
        return res.text
    
    def restart(self):
        if self.thread:
            self.thread.delete()
            self.thread = sdk.threads.create(name="Test", ttl_days=1, expiration_policy="static")

    def done(self,delete_assistant=False):
        if self.thread:
            self.thread.delete()
        if delete_assistant:
            self.assistant.delete()

In [91]:
class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки""" 
    reason : str = Field(description="Причина для вызова оператора",default="не указана")

handover = False
    
def handover_fn(x,thread):
    global handover
    handover = True
    return f"Я побежала вызывать оператора, ваш {thread.id=}, причина: {x.reason}"

In [92]:
class AddToCart(BaseModel):
    """Эта функция позволяет положить вино в корзину""" 
    wine_name : str = Field(description="Точное название вина, чтобы положить в корзину",default=None)
    count : str = Field(description="Количество бутылок вина, которое нужно положить в корзину",default=1)

carts = {}
    
def add_fn(x,thread):
    if thread.id not in carts:
        carts[thread.id] = []
    carts[thread.id].append(x)
    return f"Вино {x.wine_name} добавлено в корзину, число бутылок: {x.count}"

In [93]:
class ShowCart(BaseModel):
    """Эта функция позволяет показать содержимое корзины""" 
    
def show_fn(x,thread):
    if thread.id not in carts or len(carts[thread.id])==0:
        return "Корзина пуста"
    return "В корзине находятся следующие вина:\n" + "\n".join([
        f"{x.wine_name}, число бутылок: {x.count}" for x in carts[thread.id]])

In [94]:
instruction="""
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(instruction=instruction, search_index=index,
                   tools = {
                       SearchWinePriceList : (lambda x,t : find_wines(x)),
                       Handover : handover_fn,
                       AddToCart: add_fn,
                       ShowCart : show_fn})

In [75]:
print(wine_agent("Какое вино пьют со стейком?"))

К стейку можно подобрать разные вина в зависимости от его вида и степени прожарки:

1. **Стейк говяжий мраморный нежный (Филе-миньон)**: подойдут легкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

2. **Стейк говяжий мраморный жирноватый (Рибай и пр.)**: для прожарки Rare рекомендуются выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. Для прожарки Medium или WellDone подойдут сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки 

In [77]:
print(wine_agent("Какие вина Кьянти есть в продаже?"))

Вызываем функцию SearchWinePriceList, args=name='Кьянти' country='Италия' acidity=None color=None sort_order=None what_to_return='wine info'
В нашем магазине в продаже есть следующие вина Кьянти:

1. Вино Кверчаб Кьянти красное сухое (Италия) - 2499.0 рублей за 0,75 л.
2. Вино Полиц Кьянти красное сухое (Италия) - 1749.76 рублей за 0,75 л.
3. Вино Касал Кьянти Супериоре красное сухое (Италия) - 1349.00 рублей за 0,75 л.
4. Вино Век Кант Кьянти красное сухое (Италия) - 1099.0 рублей за 0,75 л.
5. Вино Пределла Кьянти красное сухое (Италия) - 999.0 рублей за 1,5 л. и 369.0 рублей за 0,75 л.
6. Вино Зонин Кьянти красное сухое (Италия) - 699.0 рублей за 0,75 л.


In [78]:
print(wine_agent("Добавь в корзину Полиц Кьянти, три бутылки"))

Вызываем функцию AddToCart, args=wine_name='Полиц Кьянти' count='3'
Вино Полиц Кьянти было успешно добавлено в корзину. Если вам нужно что-то еще или хотите просмотреть содержимое корзины, пожалуйста, дайте знать!


In [80]:
print(wine_agent("Ещё добавь в корзину Зонин Кьянти"))

Вызываем функцию AddToCart, args=wine_name='Зонин Кьянти' count='1'
Вино Зонин Кьянти было успешно добавлено в корзину. Если вам нужно что-то еще или хотите просмотреть содержимое корзины, пожалуйста, дайте знать!


In [81]:
print(wine_agent("Что у меня в корзине?"))

Вызываем функцию ShowCart, args=
В вашей корзине находятся следующие вина:

- Полиц Кьянти, количество бутылок: 3
- Зонин Кьянти, количество бутылок: 1


In [83]:
print(wine_agent("Вызови оператора, хочу оформить доставку!"))

Вызываем функцию Handover, args=reason='Пользователь хочет оформить доставку'
Ожидайте, скоро с вами свяжется оператор для оформления доставки.


## Многоагентное тестирование

In [95]:
instruction_user = """
Ты - простой программист, и тебе нужно выбрать вино для ужина в интернет-магазине.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть салат капрезе и стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. Говори
простым языком, короткими фразами. Когда ты удовлетворём выбором, попроси найти самое недорогое
вино данного типа, затем положи его в корзину и попроси соединить тебя с оператором, чтобы
оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    print(f"Посетитель: {msg}")
    msg = wine_agent(msg)
    print(f"Сомелье: {msg}")
    if handover:
        break
    msg = user(msg)

Посетитель: Добрый день! Поможете мне выбрать вино?
Сомелье: Добрый день! Конечно, помогу. Для начала расскажите, пожалуйста, с каким блюдом вы планируете сочетать вино или какой стиль вина вам нравится. Это поможет мне дать более точные рекомендации.
Посетитель: Добрый день! Планирую съесть салат капрезе и стейк. Хочу выбрать хорошее вино, но не очень дорогое. Посоветуете, какое подойдёт?
Сомелье: Добрый день!

Для салата Капрезе я могу порекомендовать следующие вина:
- Белые вина: игристые (брют, сухие), тихие вина — с ярким и свежим вкусом: молодые Шардоне, Рислинги, Мюллер Тургау, Грюнер Вельтлинер, Виура, «старосветский» Совиньон Блан, Пино Гри, Фалангина и т. д.
- Розовые вина: тихие лёгкие и элегантные вина Прованса (Франция) или Северной Италии.
- Игристые вина: лёгкие и элегантные белые и розовые.

Для стейка выбор вина зависит от степени его прожарки:
- Если вы предпочитаете стейк с прожаркой Medium или WellDone, то вам могут подойти сухие и полусухие вина из винограда Сира (

In [97]:
user.done(delete_assistant=True)
wine_agent.done(delete_assistant=True)

## Делаем ассистента проактивным

Мы обычно воспринимаем ассистентов как вопрос-ответных ботов, которые способны отвечать на вопросы пользователей. Однако с помощью системного промпта мы можем добавить им проактивности, чтобы они проявляли инициативу и сами инициировали диалог. Например:

In [118]:
instruction="""
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вине, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(instruction=instruction, search_index=index,
                   tools = {
                       SearchWinePriceList : (lambda x,t : find_wines(x)),
                       Handover : handover_fn,
                       AddToCart: add_fn,
                       ShowCart : show_fn})

In [119]:
print(wine_agent('/start'))

Здравствуйте! Я рад помочь вам с выбором вина. Что вы ищете? Может быть, вы планируете ужин и хотите подобрать вино к определённому блюду?

Я могу предложить вам интересное сочетание: попробуйте пино нуар из региона Хемель-эн-Аарде в ЮАР с пиццей или уткой. Это вино обладает пикантными танинами и выраженной кислотностью, которые прекрасно дополнят вкус блюд.

Если вам интересно, я могу найти это вино в нашем прайс-листе и добавить его в корзину. Также я могу помочь вам найти другие вина, которые подойдут к вашим любимым блюдам. Что вы думаете?


In [120]:
print(wine_agent('Хочу вино к стейку'))

Для стейка я могу порекомендовать несколько вариантов вин в зависимости от его прожарки и жирности:

1. Для нежного мраморного стейка (Филе-миньон) подойдут лёгкие и элегантные вина из винограда Пино Нуар, а также выдержанные варианты из сортов Неббиоло (например, Барбареско), Темпранильо (Рибейра дель Дуэро) или Санджовезе (Кьянти Ризерва).
2. Для жирноватого стейка (Рибай) с прожаркой Rare я бы порекомендовал выдержанные и благородные вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), а также супертосканские вина.
3. Если стейк прожарен Medium или WellDone, то хорошим выбором будут сухие и полусухие вина из винограда Сира (Шираз), Каберне Совиньон, Мальбек, Примитиво, Зинфандель или Альянико.

Если вас заинтересовал какой-либо из этих вариантов, я могу найти его в нашем прайс-листе и добавить в корзину. Какой вариант вам больше всего нравится?


## Делаем винного ассистента в телеграме

Для демонстрации возможностей бота мы реализуем такого ассистента в виде телеграм-бота. Конечно, для реализации полноценного телеграм-бота необходимо использовать виртуальную машину и режим webhooks, но в нашем случае мы ограничимся режимом поллинга, и запустим бота прямо в Datasphere.

> Прежде, чем запускать код ниже, необходимо создать чат-бота, пообщавшись с [@botfather](http://t.me/botfather), и разместить его секрет в виде секрета в Datasphere.

Для начала установим необходимую библиотеку:

In [103]:
%pip install telebot

I0000 00:00:1743514967.984864    3491 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [121]:
import telebot

telegram_token = os.environ['tg_token']

bot = telebot.TeleBot(telegram_token)

threads = {}

def get_thread(chat_id):
    if chat_id in threads.keys():
        return threads[chat_id]
    t = create_thread()
    print(f"New thread {t.id=} created")
    threads[chat_id] = t
    return t

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    t = get_thread(message.chat.id)
    print(f"Starting on thread {t.id=}, msg={message.text}")
    ans = wine_agent(message.text,thread=t)
    bot.send_message(message.chat.id,ans)

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    t = get_thread(message.chat.id)
    print(f"Answering on thread {t.id=}, msg={message.text}")
    answer = wine_agent(message.text,thread=t)
    bot.send_message(message.chat.id, answer)
    
# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
New thread t.id='fvtalu3ssh0vc0k7qq4q' created
Starting on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=/start
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Какое самое дешевое вино?
Вызываем функцию SearchWinePriceList, args=name=None country=None acidity=None color=None sort_order='cheapest' what_to_return='price'
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Положи вино адгум в корзину
Вызываем функцию AddToCart, args=wine_name='ВИНО АДАГУМ КБСВ КР СХ' count='1'
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Что в корзине?
Вызываем функцию ShowCart, args=
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Вызови оператора
Вызываем функцию Handover, args=reason='Пользователь хочет вызвать оператора'


In [66]:
assistant.delete()
index.delete()
for f in df['Uploaded']:
    f.delete()
for f in uploaded_foodwine:
    f.delete()

In [122]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}")
    try:
        thread.delete()
    except:
        print(" ! Error")
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

 + deleting thread id=fvtalu3ssh0vc0k7qq4q
 + deleting thread id=fvtqjo2q5dkjo9ma601b
 + deleting thread id=fvtcgiagctjp9tagg8g1
 + deleting thread id=fvt5gvhtj2ot88vk6985
 + deleting thread id=fvt1bmicj757tnptd02q
 + deleting thread id=fvt6u7tmuvcill467vu9
 + deleting thread id=fvt5ure652pa8rhb1he6
 + deleting thread id=fvts3jtade5b9esbnshb
 + deleting thread id=fvtk1n1ljsdoshhd6m98
 + deleting thread id=fvttt2b29e0j2ov9cd22
 + deleting thread id=fvtif50q8ajmjhc00egp
 + deleting thread id=fvths9v8ie27smpp23kh
 + deleting thread id=fvt3svn7g1bsu5gjgo93
 + deleting thread id=fvtgaflgjic99n81vp73
 + deleting thread id=fvts42v0dtbu2hiirq0k
 + deleting thread id=fvtbjuano57qkeb57ana
 + deleting thread id=fvt25qcm4d0l01k8r8pq
 + deleting thread id=fvt7tpsuoleau3s7lp86
 + deleting thread id=fvtka65ts1sr87bguhbc
 + deleting thread id=fvtb8n0j2it10eldqu1e
 + deleting thread id=fvt2thjoj84saa8ttesg
 + deleting thread id=fvt34koqaqj3nbp1td0a
 + deleting thread id=fvtkibt75u9fgur7ohra
 + deleting

In [145]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
print(' + Deleting files')
for file in tqdm(sdk.files.list()):
    #print(f" + deleting file id={file.id}")
    file.delete()

 + Deleting files


0it [00:00, ?it/s]
